<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Angelo Muñoz
- No fue al discord 😔


### **Link de repositorio de GitHub:** `https://github.com/FrinyildV/Entregas_MDS_7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/lab7mds/online_retail_II_cleaned.pickle'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
# df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = pd.read_pickle(path)
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
df_retail.shape

(400947, 8)

In [ ]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400947 entries, 0 to 400946
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      400947 non-null  category      
 1   StockCode    400947 non-null  category      
 2   Description  400947 non-null  object        
 3   Quantity     400947 non-null  int64         
 4   InvoiceDate  400947 non-null  datetime64[ns]
 5   Price        400947 non-null  float64       
 6   Customer ID  400947 non-null  category      
 7   Country      400947 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 15.9+ MB


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

- **Length (L)**

- **Recency (R)**

- **Monetary (M)**

- **Frequency (F)**

- **Periodicity (P)**

In [101]:

from datetime import datetime

def custom_features(dataframe_in):
    
    #hago copia para no tocar el original
    df = dataframe_in.copy()
    #ordeno
    df = df.sort_values(["Customer ID", "InvoiceDate"]).reset_index(drop=True )
    #omito la hora dado que no es relevante para las metricas
    df["fecha"]= pd.to_datetime(df['InvoiceDate']).dt.date

    # fecha más reciente
    recency = df.groupby('Customer ID')['fecha'].max().reset_index()
    recency.rename(columns={'fecha':'Last'},inplace=True)
    
    # fecha más antigua
    first = df.groupby('Customer ID')['fecha'].min().reset_index()
    first.rename(columns = {'fecha':'First'},inplace=True)
    
    # L:
    L_df=pd.merge(first,recency,on="Customer ID")
    L_df["L"] = (L_df["Last"] - L_df["First"]).dt.days

    # R: 
    recency["R"] = (recency['Last'].max()-recency['Last']).dt.days

    # F: frecuency F
    frequency = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
    frequency.rename(columns ={'Invoice': 'F'},inplace=True)
    

    # M: Monetary
    df["compra"]=df["Quantity"]*df["Price"]
    monetary = df.groupby('Customer ID')['compra'].sum().reset_index()
    monetary.rename(columns = {'compra': 'M'},inplace=True)
    
    
    # P: periodicity
    df['IVT'] = df.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    periodicity = df.groupby('Customer ID')['IVT'].std().reset_index().round(1)
    periodicity.rename(columns = {'IVT': 'P'},inplace=True)


    salida=pd.DataFrame()
    # Combinar todos los dataframes en un solo dataframe de salida
    salida = pd.merge(L_df[['Customer ID', 'L']], recency[['Customer ID', 'R']], on='Customer ID')
    salida = pd.merge(salida, frequency, on='Customer ID')
    salida = pd.merge(salida, monetary, on='Customer ID')
    salida = pd.merge(salida, periodicity, on='Customer ID')
    salida = salida.fillna(0)
    salida.set_index('Customer ID',inplace=True)
    
    return salida
    
    

In [102]:
df_lrmfp=custom_features(df_retail)
df_lrmfp.head(10)

,L,R,F,M,P
Customer ID,,,,,
12346.0,196,164,11,372.86,21.7
12347.0,37,2,2,1323.32,4.4
12348.0,0,73,1,222.16,0.0
12349.0,182,42,3,2671.14,16.2
12351.0,0,10,1,300.93,0.0
12352.0,17,10,2,343.80,3.9
12353.0,0,43,1,317.76,0.0
12355.0,0,202,1,488.21,0.0
12356.0,44,15,3,3560.30,3.7


In [103]:
df_lrmfp.isna().sum()

L    0
R    0
F    0
M    0
P    0
dtype: int64

In [65]:
df_lrmfp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4314 entries, 0 to 4313
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Customer ID  4314 non-null   category
 1   L            4314 non-null   int64   
 2   R            4314 non-null   int64   
 3   F            4314 non-null   int64   
 4   M            4314 non-null   float64 
 5   P            4166 non-null   float64 
dtypes: category(1), float64(2), int64(3)
memory usage: 373.4 KB


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [88]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
      return self
    
    def transform(self,X):
      # return X.apply(lambda x: print(x)) #entra a cada columna por separado
      return X.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)))


In [89]:
scaler = MinMax()
scaler.fit(df_lrmfp)
scaler.transform(df_lrmfp)

,L,R,F,M,P
Customer ID,,,,,
12346.0,0.525469,0.439678,0.049020,0.001068,0.119757
12347.0,0.099196,0.005362,0.004902,0.003790,0.024283
12348.0,0.000000,0.195710,0.000000,0.000636,0.000000
12349.0,0.487936,0.112601,0.009804,0.007650,0.089404
12351.0,0.000000,0.026810,0.000000,0.000862,0.000000
...,...,...,...,...,...
18283.0,0.739946,0.045576,0.024510,0.001774,0.065121
18284.0,0.000000,0.176944,0.000000,0.001322,0.000000
18285.0,0.000000,0.790885,0.000000,0.001223,0.000000


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [109]:
nombre_columnas=['L','R','M','F','P']
pipeline =  Pipeline(steps = [
     ('lrmfp',FunctionTransformer(custom_features) ), #paso 1
    ('transformacion',
     ColumnTransformer([('min_max',MinMax(),nombre_columnas)])), #paso 2
     ('tsne',TSNE(n_components = 2,random_state=1 ))

])

In [110]:
salida=pipeline.fit_transform(df_retail)
df_salida = pd.DataFrame(salida,columns=["COMP0","COMP1"])

In [111]:
fig = px.scatter(df_salida, x="COMP0",y="COMP1",
                 title="Proyección componente 1 vs componente 0")
fig

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [137]:

def aplicar_pipeline(df,k):  
  pipeline_KMEANS =  Pipeline(steps = [
      ('lrmfp',FunctionTransformer(custom_features) ), #paso 1
      ('transformacion',
      ColumnTransformer([('min_max',MinMax(),nombre_columnas)])), #paso 2
      ('kmeans',KMeans(n_clusters = k,init='k-means++',random_state=1,n_init=1 ))
  ])
  pipeline_KMEANS.fit(df)
  return pipeline_KMEANS


In [138]:

inercias = [  aplicar_pipeline(df_retail,k)['kmeans'].inertia_ for k in range(1,21) ]
inercias = pd.DataFrame({'n_cluster':range(1,21),
                         'inercia':inercias
                         })

In [140]:
inercias

,n_cluster,inercia
0,1,883.389554
1,2,357.660440
2,3,194.703394
3,4,133.635480
4,5,111.512358
5,6,95.673355
6,7,80.479320
7,8,74.453796
8,9,65.260000
9,10,59.493321


In [141]:
fig = px.line(inercias,x='n_cluster',y='inercia',title="codo K-Means")
fig

Dado lo observado, por el método del codo se elige un k=4, dado que la elasticidad de pasar al k=5 es significativamente menor que la de pasar del k=3 al k=4, es decir, la ganancia de información de aumentar el k mas allá del 4 no es significativa.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [147]:
# para el k=4 tomado:
pipeline_bueno =  aplicar_pipeline(df_retail,4) 
kmeans=pipeline_bueno['kmeans']

df_metricas=df_lrmfp.copy()
df_metricas['cluster']=kmeans.labels_
df_metricas.groupby('cluster').mean()

,L,R,F,M,P
cluster,,,,,
0,319.446828,22.760500,10.213584,5248.032609,20.118767
1,17.391066,50.898119,1.721787,672.194687,2.379310
2,17.625686,254.309550,1.504940,573.058160,2.814160
3,182.041667,66.798611,4.184524,1533.639733,20.854464


NOTA: Para este analisis  Recency se interpreta como "mientras mas alto su valor, mas interes en volver, dado que se calculó como el maximo de las ultimas compras realizadas - la ultima compra del cliente analisado, es decir, si el cliente que coincide con el maximo su ultima compra es el mismo dia es decir tiene un valor de 0 y por tanto no tiene interes en volver" 


Observando la tabla, claramente hay segmentación de clientes, donde cada cluster representa un perfil de cliente distinto. En particular los clientes del cluster 0 mantiene un periodo largo como clientes del local pero no van muy seguido (Recency bajo), mantienen una frecuencia de compras alto en todo su periodo, de media 10, y le han dado a la tienda altas sumas de dinero (Monetary alto) en el total de sus compras y aproximadamente van cada 20 dias a comprar (Periodicity).

Para el cluster 1 y 2 son clientes relativamente nuevos, cuya diferencia mas marcada es que los del cluster 2 su ultima compra fue un día lejano al día en que se realiza el estudio, por tanto son mas propensos a volver a comprar. En general estos 2 clusters tienen  Frecuencia, Monetary y Periodicity muy similares.

El cluster 3 mantiene una periodicidad equivalente a los clientes del cluster 0 pero llevan menos tiempo siendo clientes y además van con menos frecuencia a comprar, es decir, son potenciales clientes que pueden ser futuros usuarios del cluster 0 pues tienen bastante similitud.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [148]:
fig = px.scatter(df_salida, x="COMP0",y="COMP1",color=kmeans.labels_,
                 title="Proyección componente 1 vs componente 0")
fig

Se logran observar como claramente los 4 clusters estan separados en los respectivos perfiles descritos, estando más cerca los clusers 1 y 2 (naranjo y violeta) como se describio arriba. Y, a su vez, los clusters 3 y 0 (amarillo y azul) estan uno al lado del otro representando de igual forma sus similitudes. 

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>